## Lazy Model of the Hydrogen Atom with Julia

This notebook makes use of the 3-D wave equation to model the hydrogen atom. I refer to it as a lazy model because of the assumptions that I make, which can be found in the sections below. If you have any questions/comments please feel free to reach out at kylejlynch@gmail.com. Enjoy!

### Motivation

I'll start with providing some logic for why using the wave equation can be used for modeling protons and electrons.
Let's first describe the electric potential in 1, 2, and 3-dimensions using Gauss's Law. In simple terms, Gauss's Law (derived from Gauss's divergence theorem) relates what's happening inside a volume to what's happening on an arbitrary surface that encloses said volume. Specifically, if a charge $Q_{enc}$ is enclosed by an arbitrary surface $S$, then the the following holds true for electric field $\vec{E}$ 

$$\iint_{s}\vec{E} \cdot \vec{n} \; dS = \frac{Q_{enc}}{\epsilon_{0}} \tag{1}$$

which can be described as the summation of the electric field contributions through the surface $S$ is equivalent to $Q_{enc}$ divided by the permittivity of free space $\epsilon_{0}$.

For our use case we can solve for the electric field $$E = \frac{Q_{enc}}{\epsilon_{0} S} \tag{2}$$
where $S$ corresponds the the surface area of the surface surrounding $Q_{enc}$. The figure below shows the values for $S$ in 1, 2 and 3 dimensions.
<br><br>
*Note: I know the idea of "surface area" doesn't fit well in 1 and 2 dimensions, but as an example the "surface area" of a circle in 2-D is just the circumference of the circle, $2\pi r$*.

<img src="img/gauss_surfaces.png" alt="gaussian_surfaces" style="width: 350px;"/>

Note: For the 1-D case, you can derive this from the fact that the "volume" equivalent would just be $2r$, and that generally speaking $\frac{dV}{dr} = Area$, therefore $S = 2$. You can read more about this in [2] on N-Spheres (1-D is a 0-sphere).

Let's plug in the values for $S$ for each case in to $Eqn$ $2$, yielding:
$$E = \frac{Q_{enc}}{2\epsilon_{0}} \tag{3.1}$$
$$E = \frac{Q_{enc}}{2\pi\epsilon_{0}r} \tag{3.2}$$
$$E = \frac{Q_{enc}}{4\pi\epsilon_{0}r^{2}} \tag{3.3}$$
We know that the potential is $\vec{V} =- \int \vec{E} \cdot d\vec{r}$
<br><br>
Let's now calculate $V$ for each dimensionality.
$$V = -\frac{Q_{enc}}{2\epsilon_{0}} r\tag{4.1}$$
$$V = -\frac{Q_{enc}}{2\pi\epsilon_{0}} ln(r) \tag{4.2}$$
$$V = -\frac{Q_{enc}}{4\pi\epsilon_{0}r} \tag{4.3}$$

The most important take away from the result above is the proportionality between $V$ and $r$. For 1, 2, and 3 dimensions the the electric potential is
$$V \varpropto -r$$
$$V \varpropto -ln(r)$$
$$V \varpropto -\frac{1}{r}$$
respectively.

With code I'll leave in the appendix, it can be shown by adding a point source to the wave equation, that is (shown here for the 3-D case),
$$\frac{\partial ^{2}u}{\partial t^{2}} = c^{2} (\frac{\partial ^{2}u}{\partial x^{2}} + \frac{\partial ^{2}u}{\partial y^{2}} + \frac{\partial ^{2}u}{\partial z^{2}}) + \delta(x,y,z) \tag{5} ,$$

we see a similar trend for the solutions of wave equation in 1, 2 and 3-D (2-D and 3-D are cross-sections).
<img src="img/1_2_3_reg.png" alt="1_D_point" style="width: 1000px;"/>
<br><br>
**We can see that there is a symmetry between Gauss's law and the point source solutions to the wave equation.**

### Modeling the Hydrogen Atom (Lazily)

The hydrogen atom consists of two particles, one proton and one electron, which in the classical model "orbits" the proton. With more of a copenhagen quantum mechanical approach, the electron is viewed more of a probability distribution around the proton. This lazy model will be in the spirit of both the classical and quantum mechanical perspectives.
<br><br>
The proton will be modeled as a positive point source with a magnitude of 1, and the electron will be distributed around the proton in a sphere-like pattern consisting of 26 points, each having 1/26 the magnitude of the proton. The image below shows how I modeled the points (blue dots). The green lines in the figure are just to emphasize the sphere structure.
I originally built the code to do this in Python so I just pulled the screenshot from there.

<img src="img/electrons.png" alt="1_D_point" style="width: 400px;"/>

Now on to the code! Please see my other document on [Building a Numerical Solver from Scratch in Julia](https://github.com/kylejlynch/Julia/blob/main/building_numerical_solver.ipynb) if you need help understanding the code below.
<br><br>
Let's set up our variables. One of the reasons I call this lazy is due to the fact that I don't pay attention to things like the speed of light $c$ or the distance from the proton to the electron (Bohr radius), I just picked some arbitrary values. 

In [4]:
T = 20
L = 200
n_t = 200 + 1
n_x = 200 + 1
n_y = 200 + 1
n_z = 200 + 1
ts = range(0, T; length=n_t)
xs = range(-L/2, L/2; length=n_x)
ys = range(-L/2, L/2; length=n_y)
zs = range(-L/2, L/2; length=n_z)
Δt = ts[2] - ts[1]
Δx = xs[2] - xs[1]
Δy = ys[2] - ys[1]
Δz = zs[2] - zs[1]
c = 5;

In [5]:
u = zeros(n_t, n_x, n_y, n_z);

In [6]:
# Code for the electron positions
theta_list = [0,pi/4,2*pi/4,3*pi/4,4*pi/4,5*pi/4,6*pi/4,7*pi/4]
phi_list = [0,pi/4,2*pi/4,3*pi/4,4*pi/4]

function spherical_to_cartesian(r, theta, phi)
    """
    Converts spherical coordinates to cartesian, using math convention.

    Parameters
    ----------
    r : float
        radius from origin
    theta : float
        X-Y plane, 0 to 2pi.
    phi : float
        +Z to -Z, 0 to pi.

    Returns
    -------
    x, y, z

    """
    x = round(r*sin(phi)*cos(theta); digits = 4)
    y = round(r*sin(phi)*sin(theta); digits = 4)
    z = round(r*cos(phi); digits = 4)
    return x, y, z
end

coord_list = []
for theta in theta_list, phi in phi_list
    cart_tup = spherical_to_cartesian(20,theta,phi)
    trans_cart_tup = Tuple(Int(abs(round(i - 100))) for i in cart_tup)
    push!(coord_list,trans_cart_tup)
end
final_coords = unique(coord_list)

# function for the delta function
function dirac_delta(x, y, z, x_origin, y_origin, z_origin, electron_coords)
    if x == x_origin && y == y_origin && z == z_origin
        return 1
        
    elseif (x,y,z) in electron_coords
        return -1/26
    else
        return 0
    end
end

dirac_delta (generic function with 1 method)

In [7]:
# Set initial condition
for x in 2:n_x-1, y in 2:n_y-1, z in 2:n_z-1
    u[1,x,y,z] = dirac_delta(x, y, z, 100, 100, 100, final_coords) 
end

for x in 2:n_x-1, y in 2:n_y-1, z in 2:n_z-1
    u[2,x,y,z] = (
        0.5 * (c*(Δt/Δx))^2 * (u[1, x+1, y, z] - 2*u[1, x, y, z] + u[1, x-1, y, z]) 
        + 0.5 * (c*(Δt/Δy))^2 * (u[1, x, y+1, z] - 2*u[1, x, y, z] + u[1, x, y-1, z]) 
        + 0.5 * (c*(Δt/Δz))^2 * (u[1, x, y, z+1] - 2*u[1, x, y, z] + u[1, x, y, z-1]) 
        + u[1, x, y, z]
        + dirac_delta(x, y, z, 100, 100, 100, final_coords) 
    )
end

for t in 2:n_t-1, x in 2:n_x-1, y in 2:n_y-1, z in 2:n_z-1
    du_xx = (u[t, x+1, y, z] - 2*u[t, x, y, z] + u[t, x-1, y, z])/Δx^2
    du_yy = (u[t, x, y+1, z] - 2*u[t, x, y, z] + u[t, x, y-1, z])/Δy^2
    du_zz = (u[t, x, y, z+1] - 2*u[t, x, y, z] + u[t, x, y, z-1])/Δz^2
    u[t+1, x, y, z] = c^2 * (Δt^2) *(du_xx + du_yy + du_zz) + 2*u[t, x, y, z] - u[t-1, x, y, z] + dirac_delta(x, y, z, 100, 100, 100, final_coords) 
end

In [49]:
using Plots

anim = @animate for (i, t) in enumerate(ts)
    plot(
        u[i,50:150,50:150,50:150],
        xlims=(-50,50),
        ylims=(-50,50),
        zlims=(-50,50),
        label=""
    )
end
gif(anim, "wave_3d_atom.gif"; show_msg = false)
;

In [51]:
using Plots

anim = @animate for (i, t) in enumerate(ts)
    plot(
        xs,
        u[i,:,100,100];
        title = "t = $(i-1)Δt",
        legend = false,
        xlims=(-100,100),
        ylims=(-1,2),
        label=""
    )
end
gif(anim, "wave_3d_ux_atom.gif"; show_msg = false)
;

Let's emphasize the plots by comparing with what we get with just a single point source (only a proton). The "Hydrogen" atom is on the left, and the single point source is on the right.

<table>
    <tr>
        <td> <img src="img/wave_3d_atom.gif" alt="3_D_atop" style="width: 400px;"/> </td>
        <td> <img src="img/wave_3d_point.gif" alt="3_D_point" style="width: 400px;"/> </td>
    </tr>
    <tr>
        <td> <img src="img/wave_3d_ux_atom.gif" alt="3_D_ux_atom" style="width: 400px;"/> </td>
        <td> <img src="img/wave_3d_ux_point.gif" alt="3_D_point_ux" style="width: 400px;"/> </td>
    </tr>
</table>

### References
1. Briggs, William L., Lyle Cochran, and Bernard Gillett. Calculus for scientists and engineers. Pearson Education, 2014.
2. https://en.wikipedia.org/wiki/N-sphere
3. https://en.wikipedia.org/wiki/Electric_potential

### Appendix

As mentioned in the Motivation section, I generated cross sectional plots with some form of regression to show the similarities of the wave equation with Gauss's law for electrostatics. I'll provide the code for the two dimensional case here.

``` Julia
T = 20
L = 200
n_t = 200 + 1
n_x = 200 + 1
n_y = 200 + 1
ts = range(0, T; length=n_t)
xs = range(-L/2, L/2; length=n_x)
ys = range(-L/2, L/2; length=n_y)
Δt = ts[2] - ts[1]
Δx = xs[2] - xs[1]
Δy = ys[2] - ys[1]
c = 4

u = zeros(n_t, n_x, n_y);

function dirac_delta(x, y, x_point, y_point)
    if x == x_point && y == y_point
        return 1
    else
        return 0
    end
end

# Set initial condition
for x in 2:n_x-1, y in 2:n_y-1
    #u[1,x,y] = f(1,x,y)
    u[1,x,y] = dirac_delta(x, y, 100, 100) 
end

for x in 2:n_x-1, y in 2:n_y-1
    u[2,x,y] = (
        0.5 * (c*(Δt/Δx))^2 * (u[1, x+1, y] - 2*u[1, x, y] + u[1, x-1, y]) 
        + 0.5 * (c*(Δt/Δy))^2 * (u[1, x, y+1] - 2*u[1, x, y] + u[1, x, y-1]) 
        + u[1, x, y]
        + dirac_delta(x, y, 100, 100) 
    )
end

for t in 2:n_t-1, x in 2:n_x-1, y in 2:n_y-1
    du_xx = (u[t, x+1, y] - 2*u[t, x, y] + u[t, x-1, y])/Δx^2
    du_yy = (u[t, x, y+1] - 2*u[t, x, y] + u[t, x, y-1])/Δy^2
    u[t+1, x, y] = c^2 * (Δt^2) *(du_xx + du_yy) + 2*u[t, x, y] - u[t-1, x, y] + dirac_delta(x, y, 100, 100) 
end

using GLM, DataFrames, Plots
df_all = DataFrame(x=xs, y=u[end,:,100])
df_reg = DataFrame(x=xs[102:170], y=u[end,102:170,100])
model = lm(@formula(y ~ -log(x)), df_reg)
plot(df_all.x,df_all.y,label="V")
plot!(df_reg.x, predict(model, df_reg), label="logReg",linewidth=4, thickness_scaling = 1, linecolor="red")
annotate!(10, 4, text("V = -$(round(GLM.coef(model)[2];digits=2))*ln(r) + $(round(GLM.coef(model)[1];digits=2))", :red, :left, 12))
#savefig("C:\\2_D_point.png")
```